In [38]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import urllib.request as rq
import json
import os
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",100)

In [9]:
def fetch_data(index,type):
    response = rq.urlopen(os.environ["ES_URL"]+ index+'/'+type+'/_search?size=1000000')
    data = json.load(response)   
    data = data["hits"]["hits"]
    df = json_normalize(data)
    return df

## Individual conflict events

In [11]:
df = fetch_data("ucdp","event")

In [14]:
print(df.shape)
print(df.columns)

(32834, 24)
Index(['_id', '_index', '_score', '_source.best_est', '_source.ccd',
       '_source.clarity', '_source.country', '_source.d_id', '_source.d_name',
       '_source.date_end', '_source.date_start', '_source.high_est',
       '_source.is_active_conflict', '_source.latitude', '_source.longitude',
       '_source.low_est', '_source.side_a', '_source.side_a_id',
       '_source.side_b', '_source.side_b_id', '_source.source_article',
       '_source.type_of_conflict', '_source.year', '_type'],
      dtype='object')


### Looking at individual rows

In [15]:
df[5:10]

,_id,_index,_score,_source.best_est,_source.ccd,_source.clarity,_source.country,_source.d_id,_source.d_name,_source.date_end,_source.date_start,_source.high_est,_source.is_active_conflict,_source.latitude,_source.longitude,_source.low_est,_source.side_a,_source.side_a_id,_source.side_b,_source.side_b_id,_source.source_article,_source.type_of_conflict,_source.year,_type
5,AVr7VtB7u8C0brA7OyaB,ucdp,1.0,1,SOM,1,Somalia,940,Government of Somalia - Civilians,2012-11-30,2012-11-28,1,1,-0.358173,42.545365,1,Government of Somalia,95,Civilians,1,"BBC Monitoring Africa 1 December 2012 ""Program...",3,2012,event
6,AVr7VtB7u8C0brA7OyaG,ucdp,1.0,5,SOM,1,Somalia,750,Government of Somalia - Al-Shabaab,2012-02-01,2012-01-29,6,1,4.735833,45.203611,5,Government of Somalia,95,Al-Shabaab,717,"BBC Monitoring Africa 1 Feb 2012 ""Ethiopian tr...",1,2012,event
7,AVr7VtB7u8C0brA7OyaL,ucdp,1.0,12,SOM,1,Somalia,5571,Al-Shabaab - SVA,2011-04-30,2011-04-30,12,1,7.000000,48.000000,12,Al-Shabaab,717,SVA,1147,"BBC Monitoring Africa 30 April 2011 ""Fighting ...",2,2011,event
8,AVr7VtB7u8C0brA7OyaQ,ucdp,1.0,5,SOM,1,Somalia,5571,Al-Shabaab - SVA,2011-01-26,2011-01-26,5,1,4.735833,45.203611,5,Al-Shabaab,717,SVA,1147,"BBC Monitoring Africa 27 Jan 2011 ""Regional fo...",2,2011,event
9,AVr7VtB7u8C0brA7OyaV,ucdp,1.0,5,SOM,1,Somalia,5571,Al-Shabaab - SVA,2011-11-04,2011-11-04,5,1,5.038760,45.289140,3,Al-Shabaab,717,SVA,1147,"BBC Monitoring Africa 4 November 2011 ""Three k...",2,2011,event


### Finding variation in total violence for different countries

In [45]:
violence_countries = df.groupby(["_source.ccd"]).sum()["_source.high_est"]
violence_countries.values

array([ 74559,  33071,      1,   9340,   3971,   3017, 177442,  28760,
          168,   1481,  28316,  19549, 201752,  10559,   1780,    934,
         6378,  27854,   6831,     69,    445,    263,   5318,   9966,
          800,    428,   2279,  61918, 834147, 181821,   2460,  32628,
        69618,  11455,      2,  23213,    596,     59,     99,  20573,
         7188,     65,    368])

In [37]:
violence_countries.describe(percentiles=[0.6,0.8,0.9,0.99])

count        10.000000
mean     170060.275247
std      289804.009131
min           1.000000
50%       37651.341860
60%       55961.450233
80%      219735.874996
90%      595101.690000
99%      810242.469000
max      834147.000000
Name: _source.high_est, dtype: float64

In [55]:
plt.
plt.show()

TypeError: bar() missing 1 required positional argument: 'left'

## Deaths due to disease

In [20]:
ddd = fetch_data("health","deaths")

In [21]:
print(ddd.shape)
print(ddd.columns)

(1275, 7)
Index(['_id', '_index', '_score', '_source.ccode', '_source.value',
       '_source.yr', '_type'],
      dtype='object')


In [31]:
ddd[3:5]

,_id,_index,_score,_source.ccode,_source.value,_source.yr,_type
3,AVr7WUC9u8C0brA7O5Ez,health,1.0,BEN,1291.717403,1991,deaths
4,AVr7WUC9u8C0brA7O5E4,health,1.0,GAB,1219.265629,1991,deaths


### Finding variation in data

In [56]:
ddd["_source.value"].describe(percentiles=[0.2,0.9])

count    1275.000000
mean     1114.740056
std       451.388229
min       332.977111
20%       776.592083
50%      1089.457795
90%      1641.874760
max      9514.815360
Name: _source.value, dtype: float64

### Finding missing values

In [82]:
print(ddd["_source.yr"].describe())
ddd[ddd["_source.yr"]>=1989].groupby(["_source.ccode"]).count()["_source.yr"].describe()

count    1275.000000
mean     2002.000000
std         7.213932
min      1990.000000
25%      1996.000000
50%      2002.000000
75%      2008.000000
max      2014.000000
Name: _source.yr, dtype: float64


count    51.0
mean     25.0
std       0.0
min      25.0
25%      25.0
50%      25.0
75%      25.0
max      25.0
Name: _source.yr, dtype: float64

## Primary education enrollment

In [24]:
edu = fetch_data("cr","cr_country_year")

In [25]:
print(edu.shape)
print(edu.columns)

(1544, 8)
Index(['_id', '_index', '_score', '_source.c_name', '_source.ccode',
       '_source.value', '_source.yr', '_type'],
      dtype='object')


In [30]:
edu[3:5]

,_id,_index,_score,_source.c_name,_source.ccode,_source.value,_source.yr,_type
3,AVr7WbZgu8C0brA7O5Yu,cr,1.0,Angola,AGO,105.210403,2007,cr_country_year
4,AVr7WbZgu8C0brA7O5Yz,cr,1.0,Burundi,BDI,28.396400,1971,cr_country_year


### Finding missing values

In [85]:
print(edu["_source.yr"].describe())
edu[(edu["_source.yr"]>=1989) & (edu["_source.yr"]<=2014)].groupby(["_source.ccode"]).count()["_source.yr"].describe()

count    1544.000000
mean     1991.854275
std        12.763114
min      1970.000000
25%      1981.000000
50%      1991.000000
75%      2003.000000
max      2015.000000
Name: _source.yr, dtype: float64


count    43.000000
mean     20.418605
std       6.970115
min       1.000000
25%      17.000000
50%      24.000000
75%      25.000000
max      26.000000
Name: _source.yr, dtype: float64

### Finding variation in data

In [57]:
edu["_source.value"].describe(percentiles=[0.2,0.9])

count    1544.000000
mean       80.242398
std        30.187678
min         7.863490
20%        50.878506
50%        83.734425
90%       114.955193
max       149.951737
Name: _source.value, dtype: float64

## GINI coeffecient measuring inequality

In [27]:
ineq = fetch_data("ineq","ineq_country_year")

In [28]:
print(ineq.shape)
print(ineq.columns)

(299, 8)
Index(['_id', '_index', '_score', '_source.c_name', '_source.ccode',
       '_source.value', '_source.yr', '_type'],
      dtype='object')


In [32]:
ineq[3:5]

,_id,_index,_score,_source.c_name,_source.ccode,_source.value,_source.yr,_type
3,AVr7WSjGu8C0brA7O4__,ineq,1.0,Botswana,BWA,49.5,2010,ineq_country_year
4,AVr7WSjGu8C0brA7O5AB,ineq,1.0,Burkina Faso,BFA,39.0,1995,ineq_country_year


### Finding variation in data

In [58]:
ineq["_source.value"].describe(percentiles=[0.2,0.9])

count    299.000000
mean      47.356522
std       10.041831
min       28.900000
20%       39.000000
50%       45.700000
90%       60.960000
max       77.100000
Name: _source.value, dtype: float64

### Finding missing values

In [86]:
print(ineq["_source.yr"].describe())
ineq[(ineq["_source.yr"]>=1989) & (ineq["_source.yr"]<=2014)].groupby(["_source.ccode"]).count()["_source.yr"].describe()

count     299.000000
mean     1988.755853
std        17.228544
min      1914.000000
25%      1977.000000
50%      1994.000000
75%      2002.000000
max      2011.000000
Name: _source.yr, dtype: float64


count    49.000000
mean      3.795918
std       2.245176
min       1.000000
25%       2.000000
50%       4.000000
75%       5.000000
max       9.000000
Name: _source.yr, dtype: float64